# mediumの記事から持ってきたコード

URLはこちら：https://medium.com/@noahyoungs/decoding-auth-js-jwts-in-python-reverse-engineering-02deea5ce393

In [3]:
import json
import os
from typing import Any, Dict
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from jose import jwe

In [52]:
def get_derived_encryption_key(secret: str, salt: str) -> bytes:
    """
    Derives a 64-byte encryption key using HKDF for A256CBC-HS512.
    
    Args:
    secret (str): The secret key material (Auth.js secret).
    salt (str): The salt to use in key derivation, typically the name of the cookie holding the JWT.
    
    Returns:
    bytes: A 64-byte derived key.
    """
    # Define the salt for HKDF
    salt_bytes = salt.encode('utf-8')

    # Correct info string format with salt in parentheses
    info_string = f"Auth.js Generated Encryption Key ({salt})".encode('utf-8')

    # Derive a 64-byte key using HKDF for A256CBC-HS512
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=64,  # 64 bytes for A256CBC-HS512
        salt=salt_bytes,
        info=info_string
    )

    # Derive the key using the provided secret
    key = hkdf.derive(secret.encode('utf-8'))
    # Check if the derived key length is 64 bytes (512 bits)
    if len(key) != 64:
        raise ValueError("Derived key length is incorrect. Expected 64 bytes, got {} bytes.".format(len(key)))

    return key

In [53]:
def decode_authjs_session(token: str) -> Dict[str, Any]:
    print('DECODING:', token)
    
    # Fetch the Auth.js secret and salt from the environment
    authjs_secret = "secret"
    authjs_salt = "saltdesu"
    
    if not authjs_secret:
        raise ValueError("AUTHJS_SECRET environment variable is not set.")
    if not authjs_salt:
        raise ValueError("AUTHJS_SALT environment variable is not set.")
    
    # Derive the encryption key using the secret and salt
    key = get_derived_encryption_key(authjs_secret, authjs_salt)
    
    # Decrypt the token using the derived key
    try:
        data = jwe.decrypt(token, key)

        # Parse the decrypted data into JSON
        decoded_token = json.loads(data.decode('utf-8'))
        return decoded_token
    except Exception as e:
        print(f"Error during decryption: {e}")
        raise e

In [54]:
test_token = "eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwia2lkIjoiY1B4SGx3MXZGOXA1UXktRzlqSlVsVjBvZ1FZNTNPbzV1UTZwNHlNX0lFcXhnYlNacEJobjVBVnU5bmZMWmJxU0RaN0JMMU5KOFU3SnFfX0pVSHM2eGcifQ..UwJRcX9c9YLlH0_yJww7YQ.IEwxWszN5EkzWnfavqQTpOkviKu5V6PY7U4yzB-ZtP3AlF5TZr8yfO6zpN18Vr7h3ck48ZMVOdqArHL_GZtykzjxRQydSY5J6SDz_eMrWbS_xkoEiKjTUF98IlwgzUmdSRrSt9uA7igrF22Z32Ij7g.natRuwogAWvPhSjCtPicZhRuID72x9khE83RLTK98J8"
try:
    decoded_token = decode_authjs_session(test_token)
    print(f"Decoded Token: {decoded_token}")
except Exception as e:
    print(f"Token decoding failed: {e}")

DECODING: eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwia2lkIjoiY1B4SGx3MXZGOXA1UXktRzlqSlVsVjBvZ1FZNTNPbzV1UTZwNHlNX0lFcXhnYlNacEJobjVBVnU5bmZMWmJxU0RaN0JMMU5KOFU3SnFfX0pVSHM2eGcifQ..UwJRcX9c9YLlH0_yJww7YQ.IEwxWszN5EkzWnfavqQTpOkviKu5V6PY7U4yzB-ZtP3AlF5TZr8yfO6zpN18Vr7h3ck48ZMVOdqArHL_GZtykzjxRQydSY5J6SDz_eMrWbS_xkoEiKjTUF98IlwgzUmdSRrSt9uA7igrF22Z32Ij7g.natRuwogAWvPhSjCtPicZhRuID72x9khE83RLTK98J8
Decoded Token: {'sample': 'datumaa', 'iat': 1727972621, 'exp': 1730564621, 'jti': 'e2a09441-de66-4500-824a-e4064b251f72'}


In [55]:
def encode_authjs_session(obj: Dict[str, Any]) -> str:
    print('ENCODING:', obj)
    
    # Fetch the Auth.js secret and salt from the environment
    authjs_secret = "secret"
    authjs_salt = "saltdesu"
    
    if not authjs_secret:
        raise ValueError("AUTHJS_SECRET environment variable is not set.")
    if not authjs_salt:
        raise ValueError("AUTHJS_SALT environment variable is not set.")
    
    # Derive the encryption key using the secret and salt
    key = get_derived_encryption_key(authjs_secret, authjs_salt)

  
    # Decrypt the token using the derived key
    try:
        data = jwe.encrypt(json.dumps(obj), key, algorithm='dir', encryption='A256CBC-HS512')

        # Parse the decrypted data into JSON
        #encoded_token = data.encode('utf-8')
        return data.decode('utf-8')
    except Exception as e:
        print(f"Error during encryption: {e}")
        raise e

In [63]:
obj = {"himei": [345, 593]}

try:
    encoded_token = encode_authjs_session(obj)
    print(f"encoded Token: {encoded_token}")
    decoded = decode_authjs_session(encoded_token)
    print(f"Decoded Token: {decoded}")
except Exception as e:
    print(f"Token encoding failed: {e}")

ENCODING: {'himei': [345, 593]}
encoded Token: eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIn0..RP0qR5PJKWB6KyabUqZRXg.bjOZK6zDbW2yj9ZculEqRz93eDaRPiJdFpGCoJW72_k.0bIME9d8nvlyli6WLWiUk7LUlini-SzSS3262I38YaQ
DECODING: eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIn0..RP0qR5PJKWB6KyabUqZRXg.bjOZK6zDbW2yj9ZculEqRz93eDaRPiJdFpGCoJW72_k.0bIME9d8nvlyli6WLWiUk7LUlini-SzSS3262I38YaQ
Decoded Token: {'himei': [345, 593]}
